In [ ]:
%%capture

from google.colab import drive, files
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
import sys
import matplotlib.pyplot as plt
import keras
import tensorflow
import pandas as pd
from keras.layers import *
from pathlib import Path

from lstm_fcn.lstmfcn_model import *
from reading_data import load_npz_file


np.set_printoptions(edgeitems=30, linewidth=100000, formatter=dict(float=lambda x: "%.3g" % x))
GOOGLE_COLAB = "google.colab" in sys.modules
if GOOGLE_COLAB:
    sys.path.append("./gdrive/My Drive/Colab Notebooks/FRETML")
    plt.style.use("default")
    config = tensorflow.ConfigProto(device_count={"GPU": 1})
    keras.backend.set_session(tensorflow.Session(config=config))
else:
    config = tensorflow.ConfigProto(intra_op_parallelism_threads=8, inter_op_parallelism_threads=8)
    keras.backend.tensorflow_backend.set_session(tensorflow.Session(config=config))



In [ ]:
if __name__ == '__main__':
    ROOTDIR = "/Users/mag/Google Drive/Colab Notebooks/solar_flares/"
    DATADIR = "input/npz"
    OUTDIR = "output"
    DATANAME = "lstm_fcn"
    MODELNAME = "{}_best_model.h5".format(DATANAME)
    


    TRAIN = True
    NEW_MODEL = True
    SUBMISSION = False
    TEST = False

    EPOCHS = 10
    PERCENTAGE = 100
    BATCH_SIZE = 128
    CALLBACK_TIMEOUT = 15
    N_TIMESTEPS = 60  # Change if Variable length
    
    SCALER = sklearn.preprocessing.maxabs_scale
    STATIONARY = False

    if SUBMISSION:
        NPZ = 'testSet'
    else:
        NPZ = 'fold3Training'
    CSVNAME = NPZ + MODELNAME + '.csv'

    if GOOGLE_COLAB:
        ROOTDIR = "./gdrive/My Drive/Colab Notebooks" + str(ROOTDIR).split("Colab Notebooks")[-1]

    rootdir = Path(ROOTDIR)
    datadir = rootdir.joinpath(DATADIR)
    outdir = rootdir.joinpath(OUTDIR)

    X, y = load_npz_file(datadir / NPZ + '.npz')
        
    
    X = sklearn.preprocessing.StandardScaler().fit_transform(X.reshape((X.shape[0], X.shape[1]*X.shape[2]))).reshape(X.shape)

    model = keras.models.load_model(os.path.join(outdir, model_name))
    
    y_pred = model.predict(X)
    y_labels = np.argmax(y_pred,axis=1)
    
    
    if SUBMISSION:
        ids = np.arange(1, len(y_pred) + 1, dtype=int)
        df = pd.DataFrame({'Id':ids,'ClassLabel':y_labels})
        df.to_csv(outdir.join(CSVNAME))    
    else:
        y = keras.utils.to_categorical(y, num_classes=2)
        plot_confusion_matrices(
        y_target=y, y_pred=np.argmax(y_pred,axis=1), y_is_binary=True, outdir=outdir, name=DATANAME
        )
        print("F1 loss: {:.5f}".format(f1_loss(y, y_pred)))
        

